In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

/Users/chenyian/miniforge3/envs/mlp/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ONLY FOOCUS ON Mid-prices

Import Data

Data: Jan to July

(1)

Train: 1,2,3,4,5

Validation: 6

Test: 7

(2) Normalising each set

(3) Data Windowing

(4) Split window 


In [2]:
train_paths = ['/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-01.csv',
              '/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-02.csv',
              '/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-03.csv',
              '/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-04.csv',
              '/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-05.csv']

val_path = ['/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-06.csv']

test_path = ['/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-07.csv']

In [3]:
train_df = []

# Loop through the file paths and append each DataFrame to the list
for p in train_paths:
    df = pd.read_csv(p)
    train_df.append(df)

# Concatenate all DataFrames in the list
train = pd.concat(train_df, ignore_index=True)

train.head(), train.shape


(        Day     Time          Bid1        Bid2          Ask1          Ask2  \
 0  2025/1/2  7:30:00      [1.0, 6]      [0, 0]        [0, 0]        [0, 0]   
 1  2025/1/2  7:30:01    [27.0, 20]   [1.0, 12]    [798.4, 5]        [0, 0]   
 2  2025/1/2  7:30:02   [261.75, 8]  [21.0, 13]  [306.46, 13]   [403.57, 7]   
 3  2025/1/2  7:30:03  [262.75, 48]  [261.0, 8]  [292.56, 16]  [294.59, 32]   
 4  2025/1/2  7:30:04   [264.0, 36]  [261.0, 6]  [287.45, 20]   [288.1, 21]   
 
    Mid_Price  Total_volume  Bid_ask_spread  Order_imbalance  
 0       0.50             6           -1.00             1.00  
 1     412.70            37          771.40             0.73  
 2     284.11            41           44.71             0.02  
 3     277.66           104           29.81             0.08  
 4     275.73            83           23.45             0.01  ,
 (3121085, 10))

In [4]:
val = pd.read_csv('/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-06.csv')
test = pd.read_csv('/Users/chenyian/DS_mini_project/drive-download-20240302T104351Z-001/merged_2025-07.csv')

In [5]:
# mid_price prediction

train_mp = train['Mid_Price']
val_mp = val['Mid_Price']
test_mp = test['Mid_Price']

Normaling each set

In [6]:
train_mean = train_mp.mean()
train_std = train_mp.std()

train_nr = (train_mp - train_mean)/train_std
val_nr = (val_mp - train_mean)/train_std
test_nr = (test_mp - train_mean)/train_std

In [7]:
def create_sequences(data, input_sequence_length, forecast_horizon):
    X, y = [], []
    for i in range(len(data) - input_sequence_length - forecast_horizon + 1):
        X.append(data[i:(i + input_sequence_length)])
        y.append(data[(i + input_sequence_length):(i + input_sequence_length + forecast_horizon)])
    return np.array(X), np.array(y)

In [8]:
input_sequence_length = 50
forecast_horizon = 10

X_train, y_train = create_sequences(train_nr, input_sequence_length, forecast_horizon)
X_val, y_val = create_sequences(val_nr, input_sequence_length, forecast_horizon)
X_test, y_test = create_sequences(test_nr, input_sequence_length, forecast_horizon)

# Display the shape of the created sequences for verification
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

KeyboardInterrupt: 

In [ ]:
# change the shape to fit the LSTM model

X_train = X_train[:, :, np.newaxis]
X_val = X_val[:, :, np.newaxis]
X_test = X_test[:, :, np.newaxis]


y_train = y_train[:, :, np.newaxis]
y_val = y_val[:, :, np.newaxis]
y_test = y_test[:, :, np.newaxis]

# Display the shape of the created sequences for verification
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((3121026, 50, 1),
 (3121026, 10, 1),
 (642513, 50, 1),
 (642513, 10, 1),
 (30539, 50, 1),
 (30539, 10, 1))

In [ ]:
# Create tf.data.Dataset objects for the training and test sets

# Pre-batched data sets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))


train_dataset = train_dataset.batch(64)

val_dataset = val_dataset.batch(64)

test_dataset = test_dataset.batch(64)


The dataset is full of (64, 50, 1) and (64, 10, 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [ ]:
OUT_STEPS = 10
num_features = 1

lstm_model = Sequential([
    layers.LSTM(32),
    layers.Dense(OUT_STEPS*num_features, kernel_initializer = tf.initializers.zeros()),

    layers.Reshape([OUT_STEPS, num_features])
])

lstm_model.compile(loss='mse', 
              optimizer=Adam(learning_rate=0.001),
              metrics=['mean_absolute_error'])

lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)

Epoch 1/5


2024-03-17 16:34:09.757086: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-17 16:34:10.967838: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-17 16:34:11.159456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-17 16:34:11.786599: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


28987/97533 [=======>......................] - ETA: 20:25 - loss: 0.1968 - mean_absolute_error: 0.2699

KeyboardInterrupt: 